In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [185]:
seed = 1
split = 1
r = 0.33
d = 3

In [186]:
fp = f'Robust/seed{seed}/data_train_enc_{r}_{split}_Robust_d_{d}_t_3600_branching_limit_100.0_pred_tree.txt'

In [187]:
fp_data = f'../data/Warfarin_v2/rf_balance_proba/seed{seed}'

In [188]:
train_df = pd.read_csv(os.path.join(fp_data, f'data_train_enc_{r}_{split}.csv'))
test_df = pd.read_csv(os.path.join(fp_data, f'data_test_enc_{r}_{split}.csv'))

train_df_raw = pd.read_csv(os.path.join(fp_data, f'data_train_{r}_{split}.csv'))
test_df_raw = pd.read_csv(os.path.join(fp_data, f'data_test_{r}_{split}.csv'))

Process OPT results and get outcomes

In [189]:
train_df

,Age1.2,Age3.4,Age5.6,Age7,Age8.9,Height1,Height2,Height3,Height4,Height5,...,X.3..3,ml0,ml1,ml2,prob_t_pred_tree,t,y,y0,y1,y2
0,0,0,0,1,1,1,1,1,1,1,...,0,0.00,0.78,0.01,0.340667,0,0,0,1,0
1,0,1,1,1,1,1,1,1,1,1,...,0,1.00,0.02,0.00,0.340667,0,1,1,0,0
2,0,0,1,1,1,0,0,1,1,1,...,0,0.72,0.33,0.00,0.340667,0,1,1,0,0
3,0,0,0,0,1,1,1,1,1,1,...,0,1.00,0.00,0.00,0.332000,1,0,1,0,0
4,0,0,1,1,1,0,0,1,1,1,...,0,0.03,0.72,0.00,0.327333,2,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,0,1,1,1,1,1,1,1,1,...,0,0.96,0.39,0.00,0.340667,0,1,1,0,0
2996,0,0,1,1,1,1,1,1,1,1,...,0,0.42,0.78,0.00,0.327333,2,0,0,1,0
2997,0,0,1,1,1,1,1,1,1,1,...,0,1.00,0.00,0.00,0.332000,1,0,1,0,0
2998,0,0,0,1,1,0,1,1,1,1,...,0,0.99,0.57,0.00,0.340667,0,1,1,0,0


In [190]:
with open(fp) as f:
    lines = [line.rstrip() for line in f]
    
ids = []
for idx, line in enumerate(lines):
    if '#########' in line:
        ids.append(idx)

In [191]:
lines

['Set parameter TokenServer to value "hpc-licenses.hpcc.usc.edu"',
 'Set parameter TimeLimit to value 3600',
 'Set parameter TokenServer to value "hpc-licenses.hpcc.usc.edu"',
 'Set parameter TimeLimit to value 3600',
 'Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)',
 'Thread count: 20 physical cores, 20 logical processors, using up to 20 threads',
 'Optimize a model with 225030 rows, 180263 columns and 1146312 nonzeros',
 'Model fingerprint: 0x2595d23b',
 'Variable types: 180045 continuous, 218 integer (218 binary)',
 'Coefficient statistics:',
 '  Matrix range     [1e+00, 1e+00]',
 '  Objective range  [1e-02, 2e+00]',
 '  Bounds range     [1e+00, 1e+00]',
 '  RHS range        [1e+00, 1e+00]',
 'Presolve removed 53607 rows and 47577 columns (presolve time = 5s) ...',
 'Presolve removed 53608 rows and 47577 columns',
 'Presolve time: 9.50s',
 'Presolved: 171422 rows, 132686 columns, 950780 nonzeros',
 'Variable types: 132490 continuous, 196 integer (196 binary)',
 'Found heu

In [176]:
ids

[103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131]

In [196]:
# create tree structure
branching = []
leaves = []
if d == 1:
    assert len(ids) == 3
    
    branching.append(lines[ids[0]+1])
    leaves.append(lines[ids[1]+1][-1])
    leaves.append(lines[ids[2]+1][-1])
    
elif d == 2:
    assert len(ids) == 7
    for i in range(3):
        branching.append(lines[ids[i]+1])
    for i in range(3, 7):
        leaves.append(lines[ids[i]+1][-1])
        
elif d == 3:
    assert len(ids) == 15
    branching = []
    leaves = []
    for i in range(7):
        entry = lines[ids[i] + 1]
        if 'leaf' in entry:
            branching.append(f'pruned{entry[-1]}')
        else:
            branching.append(entry)
            
    for i in range(7, 15):
        entry = lines[ids[i]+1]
        if 'pruned' in entry:
            leaves.append('pruned')
        else:
            leaves.append(entry[-1])
            
    
            
    print(branching)
    print(leaves)

['X.3..3', 'VKORC1.A.A', 'Height3', 'X.2..3', 'Weight4', 'Weight3', 'pruned0']
['1', '0', '1', '0', '0', '1', 'pruned', 'pruned']


In [197]:
branching

['X.3..3', 'VKORC1.A.A', 'Height3', 'X.2..3', 'Weight4', 'Weight3', 'pruned0']

In [198]:
leaves

['1', '0', '1', '0', '0', '1', 'pruned', 'pruned']

In [202]:
def split_d1(row, branching, leaves):
    if row[branching[0]] == 0:
        return row[f'y{leaves[0]}']
    else:
        return row[f'y{leaves[1]}']
    
def split_d2(row, branching, leaves):
    if row[branching[0]] == 0:
        if row[branching[1]] == 0:
            return row[f'y{leaves[0]}']
        else:
            return row[f'y{leaves[1]}']
    else:
        if row[branching[2]] == 0:
            return row[f'y{leaves[2]}']
        else:
            return row[f'y{leaves[3]}']
        
def split_d3(row, branching, leaves):
    if row[branching[0]] == 0:
        if 'pruned' not in branching[1]:
            if row[branching[1]] == 0:
                if 'pruned' not in branching[3]:
                    if row[branching[3]] == 0:
                        return row[f'y{leaves[0]}']
                    else:
                        return row[f'y{leaves[1]}']
                else:
                    return row[f'y{branching[3][-1]}']
            else:
                if 'pruned' not in branching[4]:
                    if row[branching[4]] == 0:
                        return row[f'y{leaves[2]}']
                    else:
                        return row[f'y{leaves[3]}']
                else:
                    return row[f'y{branching[4][-1]}']
        else:
            return row[f'y{branching[1][-1]}']
    else:
        if 'pruned' not in branching[2]:
            if row[branching[2]] == 0:
                if 'pruned' not in branching[5]:
                    if row[branching[5]] == 0:
                        return row[f'y{leaves[4]}']
                    else:
                        return row[f'y{leaves[5]}']
                else:
                    return row[f'y{branching[5][-1]}']
            else:
                if 'pruned' not in branching[6]:
                    if row[branching[6]] == 0:
                        return row[f'y{leaves[6]}']
                    else:
                        return row[f'y{leaves[7]}']
                else:
                    return row[f'y{branching[6][-1]}']
        else:
            return row[f'y{branching[2][-1]}']
    
# test_df['y_dec'] = test_df.apply(lambda row: split_d1(row, branching, leaves), axis=1)
# test_df['y_dec1'] = test_df.apply(lambda row: split_d2(row, branching, leaves), axis=1)
test_df['y_dec2'] = test_df.apply(lambda row: split_d3(row, branching, leaves), axis=1)

In [141]:
def assign_d1(row, branching, leaves):
    if row[branching[0]] == 0:
        return leaves[0]
    else:
        return leaves[1]
    
def assign_d2(row, branching, leaves):
    if row[branching[0]] == 0:
        if row[branching[1]] == 0:
            return int(leaves[0])
        else:
            return int(leaves[1])
    else:
        if row[branching[2]] == 0:
            return int(leaves[2])
        else:
            return int(leaves[3])

In [121]:
test_df

,Age1.2,Age3.4,Age5.6,Age7,Age8.9,Height1,Height2,Height3,Height4,Height5,...,VKORC1.Missing,y,t,y0,y1,y2,prob_t_pred_tree,y_dec,y_logit,y_dec1
0,0,1,1,1,1,0,1,1,1,1,...,0,0,0,0,1,0,0.340667,1.0,1,1.0
1,0,1,1,1,1,0,0,0,1,1,...,0,0,2,0,1,0,0.327333,1.0,1,1.0
2,0,1,1,1,1,0,0,0,0,1,...,0,0,0,0,1,0,0.340667,1.0,1,1.0
3,0,0,1,1,1,0,0,0,1,1,...,0,0,2,0,1,0,0.327333,1.0,1,1.0
4,0,1,1,1,1,0,1,1,1,1,...,0,1,1,0,1,0,0.332000,0.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,0,0,0,1,1,1,1,1,1,1,...,0,0,0,0,1,0,0.340667,1.0,1,1.0
1382,0,1,1,1,1,0,0,0,0,1,...,0,0,1,0,0,1,0.332000,0.0,0,0.0
1383,0,0,0,0,1,0,1,1,1,1,...,0,0,2,1,0,0,0.327333,0.0,0,0.0
1384,0,0,0,0,1,0,0,1,1,1,...,0,0,0,0,1,0,0.340667,1.0,1,1.0


In [142]:
# test_df['t_d1'] = test_df.apply(lambda row: assign_d1(row, branching, leaves), axis=1)
train_df['t_d2'] = train_df.apply(lambda row: assign_d2(row, branching, leaves), axis=1)

In [132]:
train_df.columns

Index(['Age1.2', 'Age3.4', 'Age5.6', 'Age7', 'Age8.9', 'Height1', 'Height2',
       'Height3', 'Height4', 'Height5', 'Weight1', 'Weight2', 'Weight3',
       'Weight4', 'Weight5', 'Amiodarone..Cordarone.', 'Asian',
       'Black.or.African.American', 'Enzyme.Inducer', 'Unknown.Cyp2C9',
       'Unknown.Race', 'VKORC1.A.A', 'VKORC1.A.G', 'VKORC1.Missing', 'X.1..1',
       'X.1..3', 'X.2..2', 'X.2..3', 'X.3..3', 'ml0', 'ml1', 'ml2',
       'prob_t_pred_tree', 't', 'y', 'y0', 'y1', 'y2', 't_d2'],
      dtype='object')

In [134]:
train_df['nu_ki'] = train_df.apply(lambda row: row[f"ml{int(row['t'])}"], axis=1)
train_df['nu_pi'] = train_df.apply(lambda row: row[f"ml{int(row['t_d2'])}"], axis=1)

In [139]:
train_df['t'].dtype

dtype('int64')

In [143]:
train_df['t_d2'].dtype

dtype('int64')

In [144]:
(train_df['t_d2'] == train_df['t']).sum()

1006

In [145]:
train_df['dr'] = (train_df['y'] - train_df['nu_ki']) * ((train_df['t_d2'] == train_df['t'])/train_df['prob_t_pred_tree']) + train_df['nu_pi']

In [204]:
def evaluate_disparity(df_test, sensitive_group, y_opt):
    black = df_test[df_test[sensitive_group] == 1]
    nonblack = df_test[df_test[sensitive_group] == 0]
    
    return black[y_opt].mean(), nonblack[y_opt].mean()

In [205]:
evaluate_disparity(test_df, 'Black.or.African.American', 'y_dec2')

(0.8759124087591241, 0.844675740592474)

In [151]:
evaluate_disparity(train_df, 'Black.or.African.American', 'dr')

(0.8547163410925246, 0.8469449480876289)

In [148]:
train_df.columns

Index(['Age1.2', 'Age3.4', 'Age5.6', 'Age7', 'Age8.9', 'Height1', 'Height2',
       'Height3', 'Height4', 'Height5', 'Weight1', 'Weight2', 'Weight3',
       'Weight4', 'Weight5', 'Amiodarone..Cordarone.', 'Asian',
       'Black.or.African.American', 'Enzyme.Inducer', 'Unknown.Cyp2C9',
       'Unknown.Race', 'VKORC1.A.A', 'VKORC1.A.G', 'VKORC1.Missing', 'X.1..1',
       'X.1..3', 'X.2..2', 'X.2..3', 'X.3..3', 'ml0', 'ml1', 'ml2',
       'prob_t_pred_tree', 't', 'y', 'y0', 'y1', 'y2', 't_d2', 'nu_ki',
       'nu_pi', 'dr'],
      dtype='object')

In [48]:
def run(df, df_test, model):
    t_unique = df['t'].unique()
    # for 0.1, y=1 will be minority class and for 0.85, y=0 will be minority class.
    # perform smote to increase the accuracy?
    for t in t_unique:
#         print(i)
        buffer = df[df['t'] == t]
        X = buffer.iloc[:, :17]
        y = buffer['y']
        y_values = y.value_counts()

#         if y_values[1] > 5 and y_values[0] > 5:
#             smote = SMOTE(sampling_strategy=1.0, k_neighbors=5)
#         elif y_values[0] <= 5:
#             smote = SMOTE(sampling_strategy=1.0, k_neighbors=y_values[0] - 1)
#         elif y_values[1] <= 5:
#             smote = SMOTE(sampling_strategy=1.0, k_neighbors=y_values[1]-1)
#         X, y = smote.fit_resample(X, y)
#         print(y.value_counts())

        lr = model.fit(X, y)
        X_test = df_test.iloc[:, :17]
        df_test['pred' + str(t)] = [i[1] for i in lr.predict_proba(X_test)]

    ## EVALUATE PERFORMANCE
    def find_highest_y(row):
        if row['pred1'] > row['pred0'] and row['pred1'] > row['pred2']:
            return 1
        elif row['pred2'] > row['pred0'] and row['pred2'] > row['pred1']:
            return 2
        else:
            return 0

    def t_opt(row):
        if row['y1'] == 1:
            return 1
        elif row['y2'] == 1:
            return 2
        else:
            return 0

    df_test['t_opt'] = df_test.apply(lambda row: t_opt(row), axis=1)    
    df_test['t_pred'] = df_test.apply(lambda row: find_highest_y(row), axis=1)
    
    return df_test

In [49]:
test_df_raw = run(train_df_raw, test_df_raw, LogisticRegression())
test_df['y_logit'] = test_df_raw.apply(lambda row: 1 if row['t_opt'] == row['t_pred'] else 0, axis=1)

/Users/nathanjo/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/nathanjo/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

Evaluate disparity between logit and OPT

In [117]:
def evaluate_disparity(df_test, sensitive_group, y_logit, y_opt):
    black = df_test[df_test[sensitive_group] == 1]
    nonblack = df_test[df_test[sensitive_group] == 0]
    
    return black[y_logit].mean() - black[y_opt].mean(), nonblack[y_logit].mean() - nonblack[y_opt].mean()

In [111]:
sensitive_group = 'Black.or.African.American'

In [118]:
evaluate_disparity(test_df, sensitive_group, 'y_logit', 'y_dec1')

(0.007299270072992692, 0.04003202562049646)